In [4]:
import os, sys
import glob
import logging
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt

import astropy.io.fits as pyfits
import astropy.wcs as pywcs

os.environ['CRDS_PATH'] = os.path.expanduser('/lustre/JDAP/home/jdap02/crds')
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
import jwst

# grizli stuff
import grizli
from grizli import utils, prep, jwst_utils, multifit, fitting, model
from grizli.pipeline import auto_script

utils.set_warnings()

print('Grizli version: ', grizli.__version__)
print('jwst pipeline version: ', jwst.__version__)

# Quiet JWST log warnings
jwst_utils.QUIET_LEVEL = logging.WARNING
jwst_utils.set_quiet_logging(jwst_utils.QUIET_LEVEL)

Grizli version:  1.9
jwst pipeline version:  1.14.0


In [5]:
# set home path
HOME_PATH = '/lustre/JDAP/home/jdap02/start_from_s1/'
field_root = 'glass-2022data'    # the root directory for this data reduction

# first set up pipeline parameters
os.chdir(os.path.join(HOME_PATH, field_root))
kwargs = auto_script.get_yml_parameters()

sh: aws: command not found


In [6]:
####### grism forward modeling
# set pipeline parameter for this step
grism_prep_args = kwargs['grism_prep_args']

# Which filter to use as direct image?  Will try in order of the list until a match is found.
grism_prep_args['gris_ref_filters'] = {'G141': ['F140W', 'F160W'],
                                       'G102': ['F105W', 'F098M', 'F110W'],
                                       'GR150R': ['F115W', 'F150W', 'F200W'],
                                       'GR150C': ['F115W', 'F150W', 'F200W'],
                                       'GRISMR': ['F356W'],
                                       'GRISMC': ['F356W'], }

# For now, turn off refining contamination model with polynomial fits
grism_prep_args['refine_niter'] = 0
# grism_prep_args['refine_niter'] = 2

# Flat-flambda spectra
grism_prep_args['init_coeffs'] = [1.0]

grism_prep_args['mask_mosaic_edges'] = False

# Fairly bright for speedup, these can be adjusted based on how deep the spectra/visits are
grism_prep_args['refine_mag_limits'] = [23, 29]
grism_prep_args['prelim_mag_limit'] = 35

grism_prep_args['ds9'] = None

# print out relevant parameters
print('\n grism_prep_args: ', grism_prep_args)


 grism_prep_args:  {'gris_ref_filters': {'G141': ['F140W', 'F160W'], 'G102': ['F105W', 'F098M', 'F110W'], 'GR150R': ['F115W', 'F150W', 'F200W'], 'GR150C': ['F115W', 'F150W', 'F200W'], 'GRISMR': ['F356W'], 'GRISMC': ['F356W']}, 'files': None, 'grisms_to_process': None, 'split_by_grism': True, 'prelim_mag_limit': 35, 'refine_niter': 0, 'refine_poly_order': 3, 'refine_fcontam': 0.5, 'refine_mag_limits': [23, 29], 'ds9': None, 'mask_mosaic_edges': False, 'init_coeffs': [1.0]}


In [7]:
os.chdir(os.path.join(HOME_PATH, field_root, 'Prep'))
files = glob.glob('*GrismFLT.fits')

if len(files) == 0:
    ### Grism contamination model
    grp = auto_script.grism_prep(field_root=field_root, pad=800, **grism_prep_args)

    grp = multifit.GroupFLT(grism_files=glob.glob('*GrismFLT.fits'),
                            catalog='{0}-ir.cat.fits'.format(field_root),
                            cpu_count=8, sci_extn=1, pad=800)

else:
    os.chdir(os.path.join(HOME_PATH, field_root, 'Extractions'))

    grp = multifit.GroupFLT(grism_files=glob.glob('*GrismFLT.fits'),
                            catalog='{0}-ir.cat.fits'.format(field_root),
                            cpu_count=8, sci_extn=1, pad=800)

# (2024-06-27 03:59:38.269)

auto_script.grism_prep(**{'field_root': 'glass-2022data', 'PREP_PATH': '../Prep', 'EXTRACT_PATH': '../Extractions', 'ds9': None, 'refine_niter': 0, 'gris_ref_filters': {'G141': ['F140W', 'F160W'], 'G102': ['F105W', 'F098M', 'F110W'], 'GR150R': ['F115W', 'F150W', 'F200W'], 'GR150C': ['F115W', 'F150W', 'F200W'], 'GRISMR': ['F356W'], 'GRISMC': ['F356W']}, 'force_ref': None, 'files': None, 'split_by_grism': True, 'refine_poly_order': 3, 'refine_fcontam': 0.5, 'cpu_count': 0, 'mask_mosaic_edges': False, 'prelim_mag_limit': 35, 'refine_mag_limits': [23, 29], 'init_coeffs': [1.0], 'grisms_to_process': None, 'pad': 800, 'model_kwargs': {'compute_size': True}, 'sep_background_kwargs': None, 'subtract_median_filter': False, 'median_filter_size': 71, 'median_filter_central': 10, 'second_pass_filtering': False, 'box_filter_sn': 3, 'box_filter_width': 3})

auto_script.grism_prep: group = f115w
auto_script.grism_prep: N = 8
auto_script.grism_prep: file 0 = jw01324001001_

2024-06-27 11:02:57,557 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.315559 from OBSGEO-[XYZ].
2024-06-27 11:02:57,557 - stpipe - WARNING - Set OBSGEO-B to   -37.865337 from OBSGEO-[XYZ].
2024-06-27 11:02:57,557 - stpipe - WARNING - Set OBSGEO-H to 1731324282.571 from OBSGEO-[XYZ]'.
2024-06-27 11:02:57,557 - stpipe - WARNING -   warnings.warn(
2024-06-27 11:02:57,557 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.301830 from OBSGEO-[XYZ].
2024-06-27 11:02:57,557 - stpipe - WARNING - 
2024-06-27 11:02:57,557 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.296794 from OBSGEO-[XYZ].
2024-06-27 11:02:57,557 - st

jw01324001001_03101_00002_nis_rate.fits: _compute_model Done
jw01324001001_03101_00003_nis_rate.fits: _compute_model Done
jw01324001001_05101_00002_nis_rate.fits: _compute_model Done
jw01324001001_05101_00004_nis_rate.fits: _compute_model Done
jw01324001001_03101_00004_nis_rate.fits: _compute_model Done
jw01324001001_03101_00001_nis_rate.fits: _compute_model Done
jw01324001001_05101_00001_nis_rate.fits: _compute_model Done
jw01324001001_05101_00003_nis_rate.fits: _compute_model Done
Models computed - 168.40 sec.
Save jw01324001001_03101_00001_nis.01.GrismFLT.fits
Save jw01324001001_03101_00002_nis.01.GrismFLT.fits
Save jw01324001001_03101_00003_nis.01.GrismFLT.fits
Save jw01324001001_03101_00004_nis.01.GrismFLT.fits
Save jw01324001001_05101_00001_nis.01.GrismFLT.fits
Save jw01324001001_05101_00002_nis.01.GrismFLT.fits
Save jw01324001001_05101_00003_nis.01.GrismFLT.fits
Save jw01324001001_05101_00004_nis.01.GrismFLT.fits
# grism_mode_bg jw01324001001_03101_00001_nis_rate.fits GR150R 0.0

2024-06-27 11:04:59,810 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.285260 from OBSGEO-[XYZ].
2024-06-27 11:04:59,810 - stpipe - WARNING - Set OBSGEO-B to   -37.887102 from OBSGEO-[XYZ].
2024-06-27 11:04:59,810 - stpipe - WARNING - Set OBSGEO-H to 1731510579.432 from OBSGEO-[XYZ]'.
2024-06-27 11:04:59,810 - stpipe - WARNING -   warnings.warn(
2024-06-27 11:04:59,810 - stpipe - WARNING - 
2024-06-27 11:04:59,815 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.287756 from OBSGEO-[XYZ].
2024-06-27 11:04:59,815 - stpipe - WARNING - Set OBSGEO-B to   -37.885317 from OBSGEO-[XYZ].
2024-06-27 11:04:59,816 - stpipe - WARNING - Set OBSGEO-H to 1731495029.372 from OBSGEO-[XYZ]'.
2024-06-27 11:04:59,816 - stpipe - WARNING -   warnings.warn(
20

jw01324001001_09101_00002_nis_rate.fits: _compute_model Done
jw01324001001_09101_00003_nis_rate.fits: _compute_model Done
jw01324001001_09101_00004_nis_rate.fits: _compute_model Done
jw01324001001_09101_00001_nis_rate.fits: _compute_model Done
jw01324001001_11101_00001_nis_rate.fits: _compute_model Done
jw01324001001_11101_00002_nis_rate.fits: _compute_model Done
jw01324001001_11101_00004_nis_rate.fits: _compute_model Done
jw01324001001_11101_00003_nis_rate.fits: _compute_model Done
Models computed - 117.06 sec.
Save jw01324001001_09101_00001_nis.01.GrismFLT.fits
Save jw01324001001_09101_00002_nis.01.GrismFLT.fits
Save jw01324001001_09101_00003_nis.01.GrismFLT.fits
Save jw01324001001_09101_00004_nis.01.GrismFLT.fits
Save jw01324001001_11101_00001_nis.01.GrismFLT.fits
Save jw01324001001_11101_00002_nis.01.GrismFLT.fits
Save jw01324001001_11101_00003_nis.01.GrismFLT.fits
Save jw01324001001_11101_00004_nis.01.GrismFLT.fits
# grism_mode_bg jw01324001001_09101_00001_nis_rate.fits GR150R 0.0

...

Transform JWST WFSS: flip=True
Load jw01324001001_05101_00001_nis.01.GrismFLT.fits!
Transform JWST WFSS: flip=True


2024-06-27 11:10:28,814 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.301830 from OBSGEO-[XYZ].
2024-06-27 11:10:28,814 - stpipe - WARNING - Set OBSGEO-B to   -37.875226 from OBSGEO-[XYZ].
2024-06-27 11:10:28,814 - stpipe - WARNING - Set OBSGEO-H to 1731408041.458 from OBSGEO-[XYZ]'.
2024-06-27 11:10:28,815 - stpipe - WARNING -   warnings.warn(
2024-06-27 11:10:28,815 - stpipe - WARNING - 
2024-06-27 11:10:28,957 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.299300 from OBSGEO-[XYZ].
2024-06-27 11:10:28,957 - stpipe - WARNING - Set OBSGEO-B to   -37.877044 from OBSGEO-[XYZ].
2024-06-27 11:10:28,957 - stpipe - WARNING - Set OBSGEO-H to 1731423594.170 from OBSGEO-[XYZ]'.
2024-06-27 11:10:28,957 - stpipe - WARNING -   warnings.warn(
20

Load jw01324001001_17101_00003_nis.01.GrismFLT.fits!


2024-06-27 11:10:29,536 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.241840 from OBSGEO-[XYZ].
2024-06-27 11:10:29,537 - stpipe - WARNING - Set OBSGEO-B to   -37.917879 from OBSGEO-[XYZ].
2024-06-27 11:10:29,537 - stpipe - WARNING - Set OBSGEO-H to 1731786920.746 from OBSGEO-[XYZ]'.
2024-06-27 11:10:29,537 - stpipe - WARNING -   warnings.warn(
2024-06-27 11:10:29,537 - stpipe - WARNING - 
2024-06-27 11:10:30,228 - stpipe - WARNING - /lustre/JDAP/conda-env/grizli-jwst/lib/python3.11/site-packages/astropy/wcs/wcs.py:805: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.301830 from OBSGEO-[XYZ].
2024-06-27 11:10:30,228 - stpipe - WARNING - Set OBSGEO-B to   -37.875226 from OBSGEO-[XYZ].
2024-06-27 11:10:30,228 - stpipe - WARNING - Set OBSGEO-H to 1731408041.458 from OBSGEO-[XYZ]'.
2024-06-27 11:10:30,228 - stpipe - WARNING -   warnings.warn(
20

Transform JWST WFSS: flip=True
Transform JWST WFSS: flip=True
Files loaded - 34.49 sec.


In [8]:
jwst_utils.set_quiet_logging(jwst_utils.logging.WARNING)
utils.set_warnings()

_ = grp.drizzle_grism_models(root=field_root, kernel='point', scale=0.06)


glass-2022data-f115w-72.0_grism_sci.fits
# (2024-06-27 04:14:05.987)
Drizzle array 1/4
# (2024-06-27 04:14:06.585)
Drizzle array 2/4
# (2024-06-27 04:14:07.088)
Drizzle array 3/4
# (2024-06-27 04:14:07.557)
Drizzle array 4/4
glass-2022data-f115w-72.0_grism_clean.fits
# (2024-06-27 04:14:08.269)
Drizzle array 1/4
# (2024-06-27 04:14:08.725)
Drizzle array 2/4
# (2024-06-27 04:14:09.199)
Drizzle array 3/4
# (2024-06-27 04:14:09.677)
Drizzle array 4/4
glass-2022data-f115w-341.0_grism_sci.fits
# (2024-06-27 04:14:15.281)
Drizzle array 1/4
# (2024-06-27 04:14:15.939)
Drizzle array 2/4
# (2024-06-27 04:14:16.624)
Drizzle array 3/4
# (2024-06-27 04:14:17.304)
Drizzle array 4/4
glass-2022data-f115w-341.0_grism_clean.fits
# (2024-06-27 04:14:18.131)
Drizzle array 1/4
# (2024-06-27 04:14:18.786)
Drizzle array 2/4
# (2024-06-27 04:14:19.485)
Drizzle array 3/4
# (2024-06-27 04:14:20.165)
Drizzle array 4/4
glass-2022data-f200w-72.0_grism_sci.fits
# (2024-06-27 04:14:22.699)
Drizzle array 1/4
# (2024